In [1]:
import os
import numpy as np
from PIL import Image
import cv2
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical

# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
 
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [4]:
dir = 'd:\\Java\\Datasets\\Jano\\mix1\\training'

In [5]:
categories=['bikes', 'flowers', 'people']

features=[]
for i in categories:
    path=os.path.join(dir,i)
    num_classes=categories.index(i)
    for img in os.listdir(path):
        if img.endswith('.jpg'):
            
            img_array=cv2.imread(os.path.join(path,img),cv2.IMREAD_COLOR)
            img_array=cv2.resize(img_array,(150,150))
            features.append([img_array,num_classes])
X=[]
y=[]
for i,j in features:
    X.append(i)
    y.append(j)

In [7]:
X=np.array(X).reshape(-1,150,150,3)/255.0
images_train = X
labels_train = y

In [9]:
import tensorflow as tf
#from tensorflow.keras.utils import to_categorical
#y=to_categorical(y)
labels_train = tf.keras.utils.to_categorical(labels_train)
labels_train

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [10]:
from sklearn.model_selection import train_test_split
train_data,validation_data,train_labels,validation_labels=train_test_split(images_train,labels_train,test_size=0.2,random_state=12)


In [13]:
model = Sequential()

model.add(Conv2D(32, (5, 5), input_shape=(150,150,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(2, 2))

model.add(Flatten())
model.add(Dense(3))

In [14]:
epochs=10

from keras.callbacks import ReduceLROnPlateau
red_lr= ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.1)

In [15]:
model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 146, 146, 32)      2432      
_________________________________________________________________
activation_1 (Activation)    (None, 146, 146, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 71, 71, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 78400)            

In [20]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False  # randomly flip images
)


datagen.fit(train_data)

In [21]:
History = model.fit_generator(datagen.flow(train_data,train_labels, batch_size=8),
                              epochs = 20, validation_data = (validation_data,validation_labels),
                              verbose = 1)

Epoch 1/20
4/4 [==============================] - 1s 204ms/step - loss: 5.3829 - accuracy: 0.4138 - val_loss: 2.0148 - val_accuracy: 0.3750
Epoch 2/20
4/4 [==============================] - 1s 205ms/step - loss: 5.6247 - accuracy: 0.4138 - val_loss: 2.0148 - val_accuracy: 0.3750
Epoch 3/20
4/4 [==============================] - 1s 214ms/step - loss: 5.6247 - accuracy: 0.3793 - val_loss: 2.0148 - val_accuracy: 0.3750
Epoch 4/20
4/4 [==============================] - 1s 207ms/step - loss: 5.3829 - accuracy: 0.3103 - val_loss: 2.0148 - val_accuracy: 0.3750
Epoch 5/20
4/4 [==============================] - 1s 209ms/step - loss: 5.3829 - accuracy: 0.4138 - val_loss: 2.0148 - val_accuracy: 0.3750
Epoch 6/20
4/4 [==============================] - 1s 198ms/step - loss: 5.3829 - accuracy: 0.4138 - val_loss: 2.0148 - val_accuracy: 0.3750
Epoch 7/20
4/4 [==============================] - 1s 203ms/step - loss: 5.3829 - accuracy: 0.3793 - val_loss: 2.0148 - val_accuracy: 0.3750
Epoch 8/20
4/4 [====

In [27]:
train = ImageDataGenerator(rescale=1/255)
validation = ImageDataGenerator(rescale=1/255)

train_dataset = train.flow_from_directory('d:\\Java\\Datasets\\Jano\\mix2\\training',target_size=(150,150),batch_size=3)
validation_dataset = train.flow_from_directory('d:\\Java\\Datasets\\Jano\\mix2\\validation',target_size=(150,150),batch_size=3)
print(train_dataset.class_indices)

model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(16,(3,3), activation='relu',input_shape=(150,150,3)),
                                    tf.keras.layers.MaxPool2D(2,2),

                                    tf.keras.layers.Conv2D(32,(3,3), activation='relu'),
                                    tf.keras.layers.MaxPool2D(2,2),

                                    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
                                    tf.keras.layers.MaxPool2D(2, 2),

                                    tf.keras.layers.Flatten(),

                                  #  tf.keras.layers.Dense(512,activation="relu"),

                                    tf.keras.layers.Dense(6)
                                    ])

#model.compile(loss='categorical_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['accuracy'])
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model_fit = model.fit(train_dataset,steps_per_epoch=3, epochs=10,validation_data=validation_dataset)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
{'buildings': 0, 'forest': 1, 'glacier': 2, 'mountain': 3, 'sea': 4, 'street': 5}
Train for 3 steps, validate for 1000 steps
Epoch 1/10
3/3 [==============================] - 19s 6s/step - loss: 4.3823 - accuracy: 0.1111 - val_loss: 8.7951 - val_accuracy: 0.1360
Epoch 2/10
3/3 [==============================] - 18s 6s/step - loss: 8.9545 - accuracy: 0.1111 - val_loss: 8.2095 - val_accuracy: 0.1630
Epoch 3/10
3/3 [==============================] - 17s 6s/step - loss: 8.9545 - accuracy: 0.2222 - val_loss: 8.2095 - val_accuracy: 0.1710
Epoch 4/10
3/3 [==============================] - 17s 6s/step - loss: 10.7454 - accuracy: 0.2222 - val_loss: 8.2095 - val_accuracy: 0.1717
Epoch 5/10
3/3 [==============================] - 17s 6s/step - loss: 8.9545 - accuracy: 0.2222 - val_loss: 8.2095 - val_accuracy: 0.1720
Epoch 6/10
3/3 [==============================] - 17s 6s/step - loss: 10.7454 - accuracy: 0.0000e+0